In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import sys
#import cartopy.crs as crs
#import cartopy.feature as cfeature
import json
!{sys.executable} -m pip install PyAthena
import plotly
from pyathena import connect
import pandas as pd

from pyathena.async_cursor import AsyncCursor
import boto3

In [2]:
def q2df(q):
    cursor = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2', cursor_class=AsyncCursor).cursor(max_workers=2)
    query_id, future = cursor.execute(q)
    print (query_id, future)
#i=0
#for row in cursor:
#    if i==0:
#        print(row)
#    i+=1
#pd.options.display.float_format = '{:,.4f}'.format
#df = pd.read_sql(q, conn)
#df
    result_set = future.result()
    print(result_set.state)
#    print(result_set.state_change_reason)
#    print(result_set.completion_date_time)
#    print(result_set.submission_date_time)
    print('Data Scanned Bytes:', result_set.data_scanned_in_bytes)
#    print(result_set.engine_execution_time_in_millis)
#    print(result_set.query_queue_time_in_millis)
    print('Total Execution Time (ms):', result_set.total_execution_time_in_millis)
#    print(result_set.query_planning_time_in_millis)
#    print(result_set.service_processing_time_in_millis)
    print('Output location:', result_set.output_location)
    print(result_set.description)


    s3_client = boto3.client('s3')

    obj=result_set.output_location[result_set.output_location.rfind('/')+1:]
    print(obj)

    s3_client.download_file('esdis-ems-athena', obj, 'myOut.csv')

    df=pd.read_csv('myOut.csv')  
    return df    

In [3]:
q="""
select

sat_name_part,
 c.lon, 
 c.lat,
 aplv.air_pres_lay,
 aplv.so2_mol_lay
 
from 

sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)

where

c.sat_name_part in ('aqua', 'n20')

and c.obs_date_part in ('2021-08-23')

and c.asc_flag = '0'

and aplv.so2_mol_lay_qc in ('0', '1')

and aplv.air_pres_lay >= 75000

and aplv.so2_mol_lay >= 1.6e18


-- and lat >= 0.0
order by
 aplv.air_pres_lay, sat_name_part, c.lon
"""
df=q2df(q)
df

ac6a184c-4f02-4f38-b4a0-0eed702765a4 <Future at 0x7fd947122d30 state=running>
SUCCEEDED
Data Scanned Bytes: 161864439
Total Execution Time (ms): 5330
Output location: s3://esdis-ems-athena/ac6a184c-4f02-4f38-b4a0-0eed702765a4.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('air_pres_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
ac6a184c-4f02-4f38-b4a0-0eed702765a4.csv


,sat_name_part,lon,lat,air_pres_lay,so2_mol_lay
0,aqua,-179.99785,-14.958114,76561.3,1.773051e+18
1,aqua,-179.96814,-28.258764,76561.3,1.900732e+18
2,aqua,-179.94554,-75.728516,76561.3,1.612725e+18
3,aqua,-179.80002,-77.750290,76561.3,1.845275e+18
4,aqua,-179.78731,-75.095930,76561.3,1.774102e+18
...,...,...,...,...,...
492049,n20,-109.22131,-39.871437,102799.0,2.958411e+18
492050,n20,-109.08836,-41.366290,102799.0,1.823876e+18
492051,n20,-108.23507,-41.022614,102799.0,1.823837e+18
492052,n20,-107.87281,-40.105488,102799.0,1.823647e+18


In [4]:
import plotly.express as px
df['frame']=df['sat_name_part']
df['air_pres_lay'] = 0.0 - df['air_pres_lay']
fig=px.scatter_3d(df, 
           x="lon", 
           y="lat",
           z='air_pres_lay',
           animation_frame="frame",
           range_color = [ min( df['so2_mol_lay'] ), max( df['so2_mol_lay'] ) ],
           color="so2_mol_lay"
          )
fig.write_html("so2_mol_lay_3d" + ".html")

In [68]:
q="""
select
 sat_name_part,
 c.lon, 
 c.lat,
 sum(aplv.so2_mol_lay) sum_so2_mol_lay
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 c.sat_name_part in ('aqua', 'n20')
 and c.obs_date_part in ('2021-08-23')
 and c.asc_flag = '0'
 and aplv.so2_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay >= 75000
-- and aplv.so2_mol_lay >= 1.6e18
-- and lat >= 0.0
group by
 sat_name_part, c.lon, c.lat
order by
 sat_name_part, c.lon
"""
df=q2df(q)
df

8b4cd977-0f2b-4d0e-ad88-efe9a83c597d <Future at 0x7f9448aa6dd8 state=running>
SUCCEEDED
Data Scanned Bytes: 161864439
Total Execution Time (ms): 3963
Output location: s3://esdis-ems-athena/8b4cd977-0f2b-4d0e-ad88-efe9a83c597d.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('sum_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
8b4cd977-0f2b-4d0e-ad88-efe9a83c597d.csv


,sat_name_part,lon,lat,sum_so2_mol_lay
0,aqua,-179.99785,-14.958114,1.998581e+19
1,aqua,-179.99638,44.802982,1.281154e+19
2,aqua,-179.99234,6.404444,1.278301e+19
3,aqua,-179.98631,56.411210,1.286476e+19
4,aqua,-179.98620,59.051334,1.286457e+19
...,...,...,...,...
230425,n20,179.98618,-80.188350,1.122974e+19
230426,n20,179.98904,-10.270723,1.274720e+19
230427,n20,179.99026,-32.114510,2.264691e+19
230428,n20,179.99124,35.587242,1.456402e+19


In [70]:
# Sum Layers by Sat

df['size']=10

fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    animation_frame="sat_name_part",
    mapbox_style="open-street-map",
    range_color=[min(df['sum_so2_mol_lay']), max(df['sum_so2_mol_lay'])],
    color="sum_so2_mol_lay", 
    size="size",
#    color_continuous_scale='jet', 
    size_max=10, 
    hover_name='sat_name_part',
    hover_data=['lon', 'lat', 'sum_so2_mol_lay'],
    title = 'TEst')
#fig.show()
fig.write_html('sum_so2_mol_lay_map.html')

In [61]:
q="""
select 
 c.sat_name_part,
 c.asc_flag,
 aplv.air_pres_lay,
 avg(aplv.so2_mol_lay) avg_so2_mol_lay,
 stddev(aplv.so2_mol_lay) stddev_so2_mol_lay,
 count(*) num_observations
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 aplv.so2_mol_lay_qc in ('0', '1')
group by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
order by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
"""
df=q2df(q)

bcc9bffc-1ccc-4224-bdd1-c3b4b11b713c <Future at 0x7f945ba97080 state=running>
SUCCEEDED
Data Scanned Bytes: 1802657907
Total Execution Time (ms): 9800
Output location: s3://esdis-ems-athena/bcc9bffc-1ccc-4224-bdd1-c3b4b11b713c.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('asc_flag', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('air_pres_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('avg_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('stddev_so2_mol_lay', 'double', None, None, 17, 0, 'UNKNOWN'), ('num_observations', 'bigint', None, None, 19, 0, 'UNKNOWN')]
bcc9bffc-1ccc-4224-bdd1-c3b4b11b713c.csv


In [35]:
df.to_csv('so2_stats.csv')

In [36]:
df

,sat_name_part,asc_flag,air_pres_lay,avg_so2_mol_lay,stddev_so2_mol_lay,num_observations
0,aqua,0,0.283725,3.500470e+13,6.672614e+12,1778850
1,aqua,0,2.486440,1.523335e+14,2.928307e+13,1778850
2,aqua,0,5.434110,3.789430e+14,7.332354e+13,1778850
3,aqua,0,10.264200,6.800795e+14,1.322979e+14,1778850
4,aqua,0,17.533600,1.011382e+15,1.976346e+14,1778850
...,...,...,...,...,...,...
385,n20,1,89164.600000,1.516767e+18,8.635163e+16,1427017
386,n20,1,91809.800000,1.580762e+18,9.076468e+16,1390065
387,n20,1,94496.000000,1.641053e+18,9.607557e+16,1323896
388,n20,1,97223.200000,1.700826e+18,9.902583e+16,1129200


In [4]:
q="""select 
 min(aplv.so2_mol_lay) min_so2_mol_lay,
 max(aplv.so2_mol_lay) max_so2_mol_lay,
 avg(aplv.so2_mol_lay) avg_so2_mol_lay,
 stddev(aplv.so2_mol_lay) stddev_so2_mol_lay,
 count(*) num_observations
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 aplv.so2_mol_lay_qc in ('0', '1')
"""
df=q2df(q)
df

fe968232-f4e0-45a3-8896-c1d52e9225db <Future at 0x7f94609b0f60 state=running>
SUCCEEDED
Data Scanned Bytes: 1793888022
Total Execution Time (ms): 7689
Output location: s3://esdis-ems-athena/fe968232-f4e0-45a3-8896-c1d52e9225db.csv
[('min_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('max_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('avg_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('stddev_so2_mol_lay', 'double', None, None, 17, 0, 'UNKNOWN'), ('num_observations', 'bigint', None, None, 19, 0, 'UNKNOWN')]
fe968232-f4e0-45a3-8896-c1d52e9225db.csv


,min_so2_mol_lay,max_so2_mol_lay,avg_so2_mol_lay,stddev_so2_mol_lay,num_observations
0,3.363708e+12,7.105329e+18,2.435027e+17,4.081233e+17,645069560


In [9]:
q="""
with t1 as (select 
 min(aplv.so2_mol_lay) min_so2_mol_lay,
 max(aplv.so2_mol_lay) max_so2_mol_lay,
 avg(aplv.so2_mol_lay) avg_so2_mol_lay,
 stddev(aplv.so2_mol_lay) stddev_so2_mol_lay,
 count(*) num_observations
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 aplv.so2_mol_lay_qc in ('0', '1')
)
select 
 sum(case when aplv.so2_mol_lay < t1.avg_so2_mol_lay - (stddev_so2_mol_lay) then 1 end)  so2_mol_lay_lt_1std,
 sum(case when aplv.so2_mol_lay >= t1.avg_so2_mol_lay - (stddev_so2_mol_lay) and aplv.so2_mol_lay < avg_so2_mol_lay then 1 end)  so2_mol_lay_1std,
 sum(case when aplv.so2_mol_lay >= avg_so2_mol_lay and aplv.so2_mol_lay < avg_so2_mol_lay + stddev_so2_mol_lay then 1 end)  so2_mol_lay_1std,
 sum(case when aplv.so2_mol_lay >= avg_so2_mol_lay + stddev_so2_mol_lay and aplv.so2_mol_lay < avg_so2_mol_lay + (2.0 * stddev_so2_mol_lay) then 1 end)  so2_mol_lay_2std,
 sum(case when aplv.so2_mol_lay >= avg_so2_mol_lay + (2.0 * stddev_so2_mol_lay) and aplv.so2_mol_lay < avg_so2_mol_lay + (3.0 * stddev_so2_mol_lay) then 1 end)  so2_mol_lay_3std,
 sum(case when aplv.so2_mol_lay >= avg_so2_mol_lay + (3.0 * stddev_so2_mol_lay) and aplv.so2_mol_lay < avg_so2_mol_lay + (4.0 * stddev_so2_mol_lay) then 1 end)  so2_mol_lay_4std,
 sum(case when aplv.so2_mol_lay > t1.avg_so2_mol_lay + (4.0 * stddev_so2_mol_lay) then 1 end)  so2_mol_lay_gt_4std,
 count(*) num_observations
from 
 sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv),
 t1
where
 aplv.so2_mol_lay_qc in ('0', '1')
"""
df=q2df(q)
df

10e48309-8db1-45ba-96be-f22efe89560a <Future at 0x7f9459f4d748 state=running>
SUCCEEDED
Data Scanned Bytes: 3587777687
Total Execution Time (ms): 15496
Output location: s3://esdis-ems-athena/10e48309-8db1-45ba-96be-f22efe89560a.csv
[('so2_mol_lay_lt_1std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_1std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_1std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_2std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_3std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_4std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('so2_mol_lay_gt_4std', 'bigint', None, None, 19, 0, 'UNKNOWN'), ('num_observations', 'bigint', None, None, 19, 0, 'UNKNOWN')]
10e48309-8db1-45ba-96be-f22efe89560a.csv


,so2_mol_lay_lt_1std,so2_mol_lay_1std,so2_mol_lay_1std.1,so2_mol_lay_2std,so2_mol_lay_3std,so2_mol_lay_4std,so2_mol_lay_gt_4std,num_observations
0,NaN,485169706,80057246,30143841,28854509,19108712,1735546,645069560


In [10]:
q="""
with t1 as ( 
 select 
 c.sat_name_part,
 c.asc_flag,
 aplv.air_pres_lay air_pres_lay,
 avg(aplv.so2_mol_lay) avg_so2_mol_lay,
 stddev(aplv.so2_mol_lay) stddev_so2_mol_lay,
 count(*) num_observations
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 aplv.so2_mol_lay_qc in ('0', '1')
group by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
order by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
)
select 
 c.sat_name_part,
 c.obs_date_part,
 c.asc_flag,
 round(c.lon, 3) lon, 
 round(c.lat, 3) lat, 
 round(0.0 - aplv.air_pres_lay, 0) air_pres_lay,
 aplv.so2_mol_lay
from 
 sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv),
 t1
where
 c.sat_name_part = 'n20'
 and c.obs_date_part in ('2021-08-23')
 and aplv.so2_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay > 80000
 and c.sat_name_part = t1.sat_name_part
 and c.asc_flag = t1.asc_flag
 and aplv.air_pres_lay =  t1.air_pres_lay
 and aplv.so2_mol_lay > t1.avg_so2_mol_lay + (t1.stddev_so2_mol_lay * 4.0)
order by
 c.obs_date_part, asc_flag, c.sat_name_part, c.lon, 0.0 - aplv.air_pres_lay
"""
df=q2df(q)

daf8e5d3-17f6-49b4-8465-4f47b90be902 <Future at 0x7f94609f15f8 state=running>
SUCCEEDED
Data Scanned Bytes: 1016282352
Total Execution Time (ms): 7620
Output location: s3://esdis-ems-athena/daf8e5d3-17f6-49b4-8465-4f47b90be902.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('obs_date_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('asc_flag', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('air_pres_lay', 'double', None, None, 17, 0, 'UNKNOWN'), ('so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
daf8e5d3-17f6-49b4-8465-4f47b90be902.csv


In [11]:
df

,sat_name_part,obs_date_part,asc_flag,lon,lat,air_pres_lay,so2_mol_lay
0,n20,2021-08-23,0,-179.726,-31.186,-81478.0,2.205018e+18
1,n20,2021-08-23,0,-179.619,-32.656,-99991.0,2.750971e+18
2,n20,2021-08-23,0,-179.619,-32.656,-97223.0,2.654754e+18
3,n20,2021-08-23,0,-179.619,-32.656,-94496.0,2.559330e+18
4,n20,2021-08-23,0,-179.619,-32.656,-91810.0,2.464546e+18
...,...,...,...,...,...,...,...
14218,n20,2021-08-23,1,179.593,-33.056,-91810.0,2.328804e+18
14219,n20,2021-08-23,1,179.593,-33.056,-89165.0,2.231921e+18
14220,n20,2021-08-23,1,179.593,-33.056,-86561.0,2.121689e+18
14221,n20,2021-08-23,1,179.593,-33.056,-83998.0,2.012361e+18


In [12]:
import plotly.express as px
df['frame']=df['sat_name_part'] + '_' + df['asc_flag'].astype(str) + '_' + df['obs_date_part']
#df['air_pres_lay'] = 0.0 - df['air_pres_lay']
fig=px.scatter_3d(df, 
           x="lon", 
           y="lat",
           z='air_pres_lay',
           animation_frame="frame",
           range_color = [ min( df['so2_mol_lay'] ), max( df['so2_mol_lay'] ) ],
           color="so2_mol_lay"
          )
fig.write_html("climcaps_so2_mol_lay_+2_stdev" + ".html")

In [28]:
q="""
with t1 as ( 
 select 
 c.sat_name_part,
 c.asc_flag,
 aplv.air_pres_lay air_pres_lay,
 avg(aplv.so2_mol_lay) avg_so2_mol_lay,
 stddev(aplv.so2_mol_lay) stddev_so2_mol_lay,
 count(*) num_observations
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 aplv.so2_mol_lay_qc in ('0', '1')
group by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
order by
 asc_flag, c.sat_name_part, aplv.air_pres_lay
)
select 
 c.sat_name_part,
 c.obs_date_part,
 c.asc_flag,
 round(c.lon, 3) lon, 
 round(c.lat, 3) lat, 
 sum(aplv.so2_mol_lay) sum_so2_mol_lay
from 
 sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv),
 t1
where
 c.sat_name_part = 'n20'
 and c.obs_date_part in ('2021-08-23')
 and aplv.so2_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay > 90000
 and c.sat_name_part = t1.sat_name_part
 and c.asc_flag = t1.asc_flag
 and aplv.air_pres_lay =  t1.air_pres_lay
 and aplv.so2_mol_lay > t1.avg_so2_mol_lay + (t1.stddev_so2_mol_lay * 4.0)
group by 
 c.obs_date_part, c.asc_flag, c.sat_name_part, c.lon, c.lat
order by
 c.obs_date_part, c.asc_flag, c.sat_name_part, c.lon
"""
df=q2df(q)

d211f8c4-fd99-4af5-acf3-bedaa6f24a48 <Future at 0x7f94489e6940 state=running>
SUCCEEDED
Data Scanned Bytes: 1016282352
Total Execution Time (ms): 7465
Output location: s3://esdis-ems-athena/d211f8c4-fd99-4af5-acf3-bedaa6f24a48.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('obs_date_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('asc_flag', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('sum_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
d211f8c4-fd99-4af5-acf3-bedaa6f24a48.csv


In [29]:
df

,sat_name_part,obs_date_part,asc_flag,lon,lat,sum_so2_mol_lay
0,n20,2021-08-23,0,-179.619,-32.656,1.042960e+19
1,n20,2021-08-23,0,-179.447,-30.257,1.034334e+19
2,n20,2021-08-23,0,-177.665,-31.469,1.078762e+19
3,n20,2021-08-23,0,-174.283,-31.383,1.069600e+19
4,n20,2021-08-23,0,-173.985,-32.854,1.106960e+19
...,...,...,...,...,...,...
1913,n20,2021-08-23,1,179.037,-36.041,1.018920e+19
1914,n20,2021-08-23,1,179.109,-33.612,9.958562e+18
1915,n20,2021-08-23,1,179.220,-79.383,5.584772e+18
1916,n20,2021-08-23,1,179.386,-37.443,1.017553e+19


In [30]:
# Separate orbit/sat by frame

df['size']=10

fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    range_color=[min(df['sum_so2_mol_lay']), max(df['sum_so2_mol_lay'])],
    color="sum_so2_mol_lay", 
    size="size",
#    color_continuous_scale='jet', 
    size_max=10, 
    hover_name='obs_date_part',
    hover_data=['lon', 'lat', 'sum_so2_mol_lay', 'sat_name_part', 'obs_date_part'],
    title = 'TEst')
#fig.show()
fig.write_html('climcaps_sum_so2_mol_lay_+2_stdev_map.html')

In [30]:
q="""
select 
 c.sat_name_part,
 c.obs_date_part,
 c.lon, 
 c.lat, 
 c.asc_flag,
 sum(aplv.so2_mol_lay) sum_so2_mol_lay
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 c.sat_name_part in ( 'n20', 'aqua' )
 and c.obs_date_part in ('2021-08-23')
 and aplv.so2_mol_lay_qc in ('0', '1')
-- and aplv.air_pres_lay >= 85000
group by
 c.sat_name_part,
 c.obs_date_part,
 c.lon, 
 c.lat, 
 c.asc_flag
order by
 c.obs_date_part, asc_flag, c.sat_name_part, c.lon
"""
df=q2df(q)

73b80c60-20cb-47a5-9a91-01622c5d2e98 <Future at 0x7f597ad845c0 state=running>
SUCCEEDED
None
2021-12-03 17:32:33.620000+00:00
2021-12-03 17:32:25.765000+00:00
Data Scanned Bytes: 161101134
7641
164
Total Execution Time (ms): 7855
2066
50
Output location: s3://esdis-ems-athena/73b80c60-20cb-47a5-9a91-01622c5d2e98.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('obs_date_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('asc_flag', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('sum_so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
73b80c60-20cb-47a5-9a91-01622c5d2e98.csv


In [31]:
df

,sat_name_part,obs_date_part,lon,lat,asc_flag,sum_so2_mol_lay
0,aqua,2021-08-23,-179.99785,-14.958114,0,4.382915e+19
1,aqua,2021-08-23,-179.99638,44.802982,0,2.786316e+19
2,aqua,2021-08-23,-179.99352,76.369820,0,2.632140e+18
3,aqua,2021-08-23,-179.99234,6.404444,0,2.780872e+19
4,aqua,2021-08-23,-179.99055,-66.372406,0,2.632159e+18
...,...,...,...,...,...,...
647995,n20,2021-08-23,179.99384,53.257560,1,2.632129e+18
647996,n20,2021-08-23,179.99648,-60.447350,1,2.632157e+18
647997,n20,2021-08-23,179.99796,-73.400370,1,2.800231e+19
647998,n20,2021-08-23,179.99814,34.050957,1,2.957384e+19


In [33]:
# Separate orbit/sat by frame

df['size']=10

fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    animation_frame = 'frame', 
    animation_group = 'obs_date_part', 
    range_color=[min(df['sum_so2_mol_lay']), max(df['sum_so2_mol_lay'])],
    color="sum_so2_mol_lay", 
    size="size",
    color_continuous_scale='jet', 
    size_max=10, 
    hover_name='obs_date_part',
    hover_data=['lon', 'lat', 'sum_so2_mol_lay', 'sat_name_part', 'obs_date_part'],
    title = 'TEst')
#fig.show()
fig.write_html('so2_map_animation.html')

In [43]:
q="""
select
 sat_name_part,
 c.lon, 
 c.lat,
 aplv.air_pres_lay,
 aplv.so2_mol_lay
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 c.sat_name_part in ('aqua', 'n20')
 and c.obs_date_part in ('2021-08-23')
 and c.asc_flag = '0'
 and aplv.so2_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay >= 80000
order by
 aplv.air_pres_lay, sat_name_part, c.lon
"""
df=q2df(q)

4ed1303e-9faa-44d4-ad2a-c0c9480f1eff <Future at 0x7f59aa106ef0 state=running>
SUCCEEDED
None
2021-12-03 17:46:53.296000+00:00
2021-12-03 17:46:39.620000+00:00
Data Scanned Bytes: 161864439
8097
5541
Total Execution Time (ms): 13676
473
38
Output location: s3://esdis-ems-athena/4ed1303e-9faa-44d4-ad2a-c0c9480f1eff.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('air_pres_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('so2_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
4ed1303e-9faa-44d4-ad2a-c0c9480f1eff.csv


In [44]:
df

,sat_name_part,lon,lat,air_pres_lay,so2_mol_lay
0,aqua,-179.99785,-14.958114,81477.5,2.011217e+18
1,aqua,-179.99638,44.802982,81477.5,1.288330e+18
2,aqua,-179.99234,6.404444,81477.5,1.288048e+18
3,aqua,-179.98631,56.411210,81477.5,1.291198e+18
4,aqua,-179.98620,59.051334,81477.5,1.291479e+18
...,...,...,...,...,...
1505239,n20,-109.22131,-39.871437,102799.0,2.958411e+18
1505240,n20,-109.08836,-41.366290,102799.0,1.823876e+18
1505241,n20,-108.23507,-41.022614,102799.0,1.823837e+18
1505242,n20,-107.87281,-40.105488,102799.0,1.823647e+18


In [45]:
# Frame by pressure

df['size']=10
df['frame']=df['air_pres_lay'].astype(str) + '_' + df['sat_name_part']
fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    animation_frame = 'frame', 
#    animation_group = 'obs_date_part', 
    range_color = [ min( df['so2_mol_lay'] ), max( df['so2_mol_lay'] ) ],
    color="so2_mol_lay", 
    size="size",
    color_continuous_scale='jet', 
    size_max=10, 
    title = 'so2 by pressure layers'
)
#fig.show()
fig.write_html('so2_layer_map_animation.html')

In [12]:
# Similar but with co2

q="""
select 
 c.sat_name_part,
 c.obs_date_part,
 c.lon, 
 c.lat, 
 c.asc_flag,
 clim_co2_mmr,
 sum(apv.co2_vmr) sum_co2_vmr
from 
sci.climcaps c cross join unnest (c.air_pres_vars) t(apv)
where
 c.sat_name_part = 'n20'
 and c.obs_date_part in ('2021-08-23')
 and apv.co2_vmr_qc in ('0', '1')
-- and apv.air_pres >= 85000
-- and apv.air_pres < 85000
group by
 c.sat_name_part,
 c.obs_date_part,
 c.lon, 
 c.lat, 
 c.asc_flag,
 c.clim_co2_mmr
order by
 c.obs_date_part, asc_flag, c.sat_name_part, c.lon
"""
df=q2df(q)

98b6f938-c082-4732-9440-3f3adeefd324 <Future at 0x7f9c8a05af98 state=running>
SUCCEEDED
None
2021-12-03 14:38:29.549000+00:00
2021-12-03 14:38:25.161000+00:00
Data Scanned Bytes: 95893239
4191
178
Total Execution Time (ms): 4388
None
19
Output location: s3://esdis-ems-athena/98b6f938-c082-4732-9440-3f3adeefd324.csv
[('sat_name_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('obs_date_part', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('asc_flag', 'varchar', None, None, 2147483647, 0, 'UNKNOWN'), ('clim_co2_mmr', 'float', None, None, 17, 0, 'UNKNOWN'), ('sum_co2_vmr', 'float', None, None, 17, 0, 'UNKNOWN')]
98b6f938-c082-4732-9440-3f3adeefd324.csv


In [13]:
df

,sat_name_part,obs_date_part,lon,lat,asc_flag,clim_co2_mmr,sum_co2_vmr
0,n20,2021-08-23,-179.99982,75.681500,0,0.000395,0.039462
1,n20,2021-08-23,-179.99828,-51.427830,0,0.000395,0.039669
2,n20,2021-08-23,-179.99660,-37.447895,0,0.000395,0.039807
3,n20,2021-08-23,-179.99435,-19.487148,0,0.000395,0.039920
4,n20,2021-08-23,-179.99376,43.474796,0,0.000395,0.039467
...,...,...,...,...,...,...,...
323995,n20,2021-08-23,179.99384,53.257560,1,0.000395,0.024673
323996,n20,2021-08-23,179.99648,-60.447350,1,0.000395,0.023515
323997,n20,2021-08-23,179.99796,-73.400370,1,0.000395,0.039397
323998,n20,2021-08-23,179.99814,34.050957,1,0.000395,0.039942


In [82]:
df['size']=10
df['frame']=df['sat_name_part'] + '_' + df['asc_flag'].astype(str) + '_' + df['obs_date_part']
fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    animation_frame = 'frame', 
    animation_group = 'obs_date_part', 
    color="sum_co2_vmr", 
    size="size",
    color_continuous_scale='jet', 
    size_max=10, 
    hover_name='obs_date_part',
    hover_data=['lon', 'lat', 'sum_co2_vmr'],
    title = 'co2 sum all pressure layers')
#fig.show()
fig.write_html('co2_map_animation.html')

ValueError: Value of 'hover_data_2' is not the name of a column in 'data_frame'. Expected one of ['sat_name_part', 'obs_date_part', 'asc_flag', 'lon', 'lat', 'air_pres_lay', 'so2_mol_lay', 'frame', 'size'] but received: sum_co2_vmr

In [16]:
q="""
select 
 c.lon, 
 c.lat, 
 aplv.air_pres_lay,
 aplv.ch4_mol_lay
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv)
where
 c.sat_name_part = 'n20'
 and c.obs_date_part in ('2021-08-23')
 and aplv.ch4_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay >= 85000
 and asc_flag = '1'
order by
 aplv.air_pres_lay, c.lon
"""
df=q2df(q)

babf48e5-c1e1-4cce-aee8-51ea139b3fa5 <Future at 0x7f945843d898 state=running>
SUCCEEDED
Data Scanned Bytes: 117861986
Total Execution Time (ms): 4505
Output location: s3://esdis-ems-athena/babf48e5-c1e1-4cce-aee8-51ea139b3fa5.csv
[('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('air_pres_lay', 'float', None, None, 17, 0, 'UNKNOWN'), ('ch4_mol_lay', 'float', None, None, 17, 0, 'UNKNOWN')]
babf48e5-c1e1-4cce-aee8-51ea139b3fa5.csv


In [17]:
df

,lon,lat,air_pres_lay,ch4_mol_lay
0,-179.99908,-18.339355,86560.8,1.004257e+22
1,-179.99745,-16.379068,86560.8,9.941931e+21
2,-179.99260,-20.297726,86560.8,9.764963e+21
3,-179.99165,73.209370,86560.8,1.046975e+22
4,-179.99042,-28.122002,86560.8,1.012102e+22
...,...,...,...,...
609349,179.97162,37.555042,99991.0,1.102143e+22
609350,179.97550,48.680878,99991.0,1.117014e+22
609351,179.97841,-30.077648,99991.0,1.086428e+22
609352,179.98220,-50.333107,99991.0,1.055513e+22


In [18]:
df['size']=10
df['frame']=df['air_pres_lay'].astype(str) 
fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    animation_frame = 'frame', 
#    animation_group = 'obs_date_part', 
    range_color = [ min( df['ch4_mol_lay'] ), max( df['ch4_mol_lay'] ) ],
    color="ch4_mol_lay", 
    size="size",
    color_continuous_scale='jet', 
    size_max=10, 
    title = 'ch4 by pressure layers'
)
#fig.show()
fig.write_html('ch4_layer_map_animation.html')

In [26]:
q="""
select 
 c.lon, 
 c.lat, 
 apv.air_pres,
 apv.co2_vmr
from 
sci.climcaps c cross join unnest (c.air_pres_vars) t(apv)
where
 c.sat_name_part = 'n20'
 and c.obs_date_part in ('2021-08-26')
 and c.asc_flag = '1'
 and apv.co2_vmr_qc in ('0', '1')
 and apv.air_pres >= 85000
-- and apv.air_pres < 85000
order by
 apv.air_pres, c.lon
"""
df=q2df(q)

e08e7dd4-4ad9-41de-9c59-bd1460029d12 <Future at 0x7f1e14ae0550 state=running>
SUCCEEDED
None
2021-11-30 17:01:50.911000+00:00
2021-11-30 17:01:45.377000+00:00
Data Scanned Bytes: 96017811
5223
194
Total Execution Time (ms): 5534
480
117
Output location: s3://esdis-ems-athena/e08e7dd4-4ad9-41de-9c59-bd1460029d12.csv
[('lon', 'float', None, None, 17, 0, 'UNKNOWN'), ('lat', 'float', None, None, 17, 0, 'UNKNOWN'), ('air_pres', 'float', None, None, 17, 0, 'UNKNOWN'), ('co2_vmr', 'float', None, None, 17, 0, 'UNKNOWN')]
e08e7dd4-4ad9-41de-9c59-bd1460029d12.csv


In [27]:
df

,lon,lat,air_pres,co2_vmr
0,-179.99594,19.282198,85278.82,0.000408
1,-179.99496,-68.738190,85278.82,0.000408
2,-179.99370,23.708158,85278.82,0.000408
3,-179.98993,-28.774052,85278.82,0.000407
4,-179.98816,27.647783,85278.82,0.000409
...,...,...,...,...
703379,179.97787,-34.734550,101394.78,0.000407
703380,179.98401,29.612114,101394.78,0.000409
703381,179.98616,35.026060,101394.78,0.000409
703382,179.99352,-48.205070,101394.78,0.000409


In [28]:
df['size']=10
df['frame']=df['air_pres'].astype(str) 
fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon",
    mapbox_style="open-street-map",
    animation_frame = 'frame', 
#    animation_group = 'obs_date_part', 
    range_color = [ min( df['co2_vmr'] ), max( df['co2_vmr'] ) ],
    color="co2_vmr", 
    size="size",
    color_continuous_scale='jet', 
    size_max=10, 
    title = 'co2 by pressure layers'
)
#fig.show()
fig.write_html('co2_layer_map_animation.html')

In [ ]:
q="""
select 
 c.sat_name_part,
 c.obs_date_part,
 c.lon, 
 c.lat, 
 c.asc_flag,
 date_parse(cast(c.obs_time_utc[1] as varchar) ||'-'|| cast(c.obs_time_utc[2] as varchar) ||'-'|| cast(c.obs_time_utc[3] as varchar) ||'T'|| cast(c.obs_time_utc[4] as varchar) ||':'|| cast(c.obs_time_utc[5] as varchar), '%Y-%c-%eT%k:%i'),
 0.0 - aplv.air_pres_lay air_pres_lay,
 aplv.mw_h2o_vap_mol_lay,
 aplv.mw_h2o_vap_mol_lay - avg_of_measure  diff_from_mean
from 
sci.climcaps c cross join unnest (c.air_pres_lay_vars) t(aplv),
climcaps_work_area.climcaps_stats_10d s
where
 c.sat_name_part = 'n20'
 and s.measurement_name = 'mw_h2o_vap_mol_lay'
 and c.obs_date_part in ('2021-08-27', '2021-08-28', '2021-08-29')
 and c.lon between s.lon - 5 and s.lon + 5
 and c.lat between s.lat - 5 and s.lat + 5
 and c.obs_time_utc[2] = s.month
 and aplv.mw_h2o_vap_mol_lay_qc in ('0', '1')
 and aplv.air_pres_lay >= 95000.0
 and aplv.air_pres_lay = s.air_pres_lay
 and round( abs( aplv.mw_h2o_vap_mol_lay - avg_of_measure ) / stddev_of_measure, 1) >= 2.5
order by
 c.obs_date_part, asc_flag, c.sat_name_part, c.lon, 0.0 - aplv.air_pres_lay
"""
df=q2df(q)

In [ ]:
import plotly.express as px
df['frame']=df['sat_name_part'] + '_' + df['asc_flag'].astype(str) + '_' + df['obs_date_part']
df['text']=df['mw_h2o_vap_mol_lay'].astype(str)
fig=px.scatter_3d(df, 
           x="lon", 
           y="lat",
           z='air_pres_lay',
           animation_frame="frame", 
           color="diff_from_mean"
          )
fig.write_html("climcaps_sigma2_4plus_global_animation" + ".html")